In [43]:
import numpy as _np
import spinmob as _s
import matplotlib.pyplot as _plt
import os as _os
from scipy.optimize import curve_fit

In [95]:
data_box = _s.data.load(path = "problem1data.csv")
data_box2 = _s.data.load(path = "problem1data_zoomed.csv")


In [45]:
def exp_decay(x, gamma, amp):
    return amp*_np.exp(-x/gamma)

def correlation(data, data_average):
    data_correlations = []
    correlation_distance = _np.array(range(len(data)-1))+1
    for distance in correlation_distance:
        correlations = data[:-distance]*data[distance::]-data_average**2
        data_correlations.append(_np.average(correlations))
    return data_correlations, correlation_distance

def fit_exp_decay(data, distacnes, p0):
    popt, cov = curve_fit(exp_decay, distacnes[:2000], data[:2000], p0=p0)
    return [popt, cov]

def plot_exp_decay_fit(correlation, distances, popt, ep_str, n_or_e):
    _plt.plot(distances, correlation, label="data")
    _plt.axhline(0, linestyle="--", color="grey", alpha=0.5)
    _plt.plot(distances[:2000], exp_decay(distances[:2000], *popt), label="fit")
    _plt.title(n_or_e+" correlation for ε="+ep_str, size=35)
    _plt.ylabel(r'$\tilde{e_n}\tilde{e}_{n+d} - \langle \tilde{e}\rangle^2$', size=35)
    _plt.xlabel("d", size=35)
    _plt.tick_params(axis="both", labelsize=30)
    _plt.legend(fontsize=30)
    _plt.show()

def choose_unc(es, es_popt, ns, ns_popt):
    es_uncs = []
    ns_uncs = []
    num_trials = len(es)
    for position, block in enumerate(block_sizes):
        blocked_es = _np.reshape(es, (int(num_trials/block), block))
        block_avg_es = _np.average(blocked_es, axis=1)
        unc = _np.std(block_avg_es)/_np.sqrt(len(block_avg_es))
        es_uncs.append(unc)

        blocked_ns = _np.reshape(ns, (int(num_trials/block), block))
        block_avg_ns = _np.average(blocked_ns, axis=1)
        unc = _np.std(block_avg_ns)/_np.sqrt(len(block_avg_ns))
        ns_uncs.append(unc)

        print("E Unc: ", es_uncs[-1], "E correlation/unc: ", _np.sqrt(exp_decay(block, *es_popt))/unc)
        print("n Unc: ", ns_uncs[-1], "n correlation/unc: ", _np.sqrt(exp_decay(block, *ns_popt))/unc)
        print(" num blocks: ", len(block_avg_es), " blocksize: ", block, "choice number", position)
        print("\r\r\r")
    which_block = int(input("Which block_size?"))
    return [block_sizes[which_block], es_uncs[which_block], ns_uncs[which_block]]


In [96]:
mus, ns, es, ws = _np.array([data for data in data_box])
mus2, ns2, es2, ws2 = _np.array([data for data in data_box2])
mus = _np.append(mus, mus2)
ns = _np.append(ns, ns2)
es = _np.append(es, es2)
ws = _np.append(ws, ws2)

In [108]:
indexes = _np.nonzero(mus==list(set(mus))[10])
ns0 = ns[indexes]

In [109]:
indexes = _np.nonzero(mus==list(set(mus))[2])
ns0 = ns[indexes]
ns0_average = _np.average(ns0)
ns0_correlation, ns0_distances = correlation(ns0, ns0_average)
p0 = [200, 30]
ns0_popt, ns0_cov = fit_exp_decay(ns0_correlation, ns0_distances, p0)
p0 = [50, 800]
ns0_popt, ns0_cov = fit_exp_decay(ns0_correlation, ns0_distances, p0)
plot_exp_decay_fit(ns0_correlation, ns0_distances, ns0_popt, "", r"$n$")
ns0_popt

array([3.63435139, 0.19710816])

In [116]:
block_size = 20
equilibriation_time = 25
exp_decay(block_size, *ns0_popt)/ns0_average*100

0.06346170557600839

In [100]:
mus_final = []
ns_final = []
ns_unc_final = []
N=20
for mu in set(mus):
    mus_final.append(mu)
    indexes = _np.nonzero(mus==mu)
    ns_temp = ns[indexes][equilibriation_time:]
    ns_final.append(_np.average(ns_temp))
    ns_unc_final.append(_np.std(ns_temp) / _np.sqrt(len(ns_temp)/block_size))
sort_indexes = _np.argsort(mus_final)
mus_final = _np.array(mus_final)[sort_indexes]
ns_final = _np.array(ns_final)[sort_indexes]
ns_unc_final = _np.array(ns_unc_final)[sort_indexes]

In [103]:
_plt.errorbar(mus_final, ns_final, yerr=ns_unc_final, marker=".", color="blue", zorder=1)
_plt.errorbar(mus_final, ns_final, yerr=ns_unc_final, marker="", linestyle="", color="red", zorder=2, label="error bar")
_plt.title(r"$n \: vs \: \mu \: at \: N=4, \: \beta=100, \epsilon=0.01$", size=18)
_plt.xlabel(r"$\mu$", size=18)
_plt.ylabel(r"$n$", size=18)
_plt.legend()

In [114]:
for jj in range(len(mus_final)):
    if (jj+1)%20==0:
        print(mus_final[jj], ns_final[jj], ns_unc_final[jj])

0.025252525252525256 0.925371142618849 0.006788077220026601
0.05050505050505051 0.9939949958298582 0.001995648994470182
0.13131313131313133 1.1213344453711427 0.008434109931325965
0.15656565656565657 1.6399332777314428 0.012399216467034796
0.20202020202020204 1.9943619683069225 0.001934066537707664
0.29292929292929293 2.2155462885738113 0.01062158521271363
0.31818181818181823 2.777347789824854 0.010746241377304677
0.3535353535353536 2.9916263552960802 0.0023537844924133495
0.45 3.1706422018348626 0.009717389064647787
0.47586206896551725 3.7362468723936613 0.011382717106652689


In [12]:
for jj in range(len(chis)):
    print(betas_final[jj], chis[jj], chis_unc[jj])

0.75 0.00412 0.002264681434551006
0.775 0.001912006321490562 0.0018054096892704218
0.8 0.00838720608052631 0.0031465688919556007
0.825 0.017815580962482924 0.004740249204437283
0.85 0.05427011183033771 0.008571369630605234
0.875 0.11166822459615378 0.012918781067399672
0.9 0.30659283188035263 0.023592701265924724
0.925 0.5169098470292232 0.03281467842691488
0.95 0.9357766803016367 0.05004449136966139
0.975 1.2070320504675711 0.06294235438385021
1.0 1.5386741561593251 0.07618842747371263
1.025 1.7865229573588368 0.08969996425442045
1.05 2.0175173894353673 0.10117626794278058
1.075 2.256677930311366 0.1107473751545197
1.1 2.4793002040418837 0.1228539133345776
